In [39]:
import numpy as np
import random

In [40]:
class neuron():
    def __init__(self,bias):
        self.bias=bias
        self.weights=[]

    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def sum_input_to_hiddedn(self):
        sum=0
        for i in range(len(self.input)):
            sum += self.input[i]*self.weights[i]
            # print(sum)
        return sum+self.bias
    
    

    def cost(self,input):
        self.input=input
        self.output=self.sigmoid(self.sum_input_to_hiddedn())
        return self.output
    
    def cal_error(self,target_op):
        return 0.5*np.square(target_op-self.output)
    
    def error_wrt_output(self, targer_op):
        return -(targer_op-self.output)


    def error_wrt_input(self):
        return self.output*(1-self.output)

    def total_error(self,targer_op):
        return self.error_wrt_input()*self.error_wrt_output(targer_op)
    

    # def error_wrt_weight(self,index):
    #     return self.input[index]

    

In [41]:
class network():
    def __init__(self,hidden_layer,bias):
        self.network=[]
        self.bias = bias if bias is not None else random.random()
        for _ in range(hidden_layer):
            self.network.append(neuron(self.bias))

    def forward(self,input):
        outputs=[]
        for i in self.network:
            outputs.append(i.cost(input))
        return outputs
    

    # def backPropogation(self,):
        

In [42]:
class all_network():
    def __init__(self,n_input_layer,n_hidden_layer,n_output_layer,hidden_weights,output_weights,hidden_bias,output_bias):
        self.hidden_l_network=network(n_hidden_layer,hidden_bias)
        self.output_l_network=network(n_output_layer,output_bias)
        self.n_input_layer=n_input_layer
        # self.hidden_weight_layer=self.add_weights_to_hidden(hidden_weights)
        # self.output_weight_layer=self.add_weights_to_output(output_weights)
        self.add_weights(self.hidden_l_network,hidden_weights,n_input_layer)
        self.add_weights(self.output_l_network,output_weights,n_hidden_layer)


    def add_weights(self, layer, weight, num_inputs):
        count = 0
        for neuron in layer.network:
            for _ in range(num_inputs):
                neuron.weights.append(weight[count])
                count += 1

    # def add_weights_to_output(self,output_weights):
    #     count=0
    #     for i in range(len(self.output_l_network.network)):
    #         # print(hidden_l_network.network[i].weight)
    #         for j in range((self.n_input_layer)):
    #             self.output_l_network.network[i].weight.append(output_weights[count])
    #             count+=1

    def forward_hidden_op(self,input_data):
        hidden_op=self.hidden_l_network.forward(input_data)
        # self.hidden_l_cost=self.hidden_l_network.forward(input_data)
        # print("Hidden layer outputs:", self.hidden_l_cost)
        # self.output_l_cost=self.output_l_network.forward(self.hidden_l_cost)
        # print("Output layer outputs:", self.output_l_cost)  # Print output layer outputs

        return self.output_l_network.forward(hidden_op)
    
    def backPropogation(self, input_data, target_op,learning_rate=0.5):
        # Output layer deltas
        # self.forward_hidden_op(target_inp)
        hidden_output = self.hidden_l_network.forward(input_data)
        final_output = self.output_l_network.forward(hidden_output)

        # output_deltas = [0] * len(self.output_l_network.network)
        # for i in range(len(self.output_l_network.network)):
        #     output_deltas[i] = self.output_l_network.network[i].cal_error(target_op[i])

        output_deltas = []
        for i in range(len(target_op)):
            error_wrt_input = self.output_l_network.network[i].error_wrt_input()
            output_deltas.append((target_op[i] - final_output[i]) * error_wrt_input)

        # Hidden layer deltas
        hidden_deltas = []
        for i in range(len(self.hidden_l_network.network)):
            # Calculate the weighted sum of output deltas for the current hidden layer neuron
            weighted_sum = 0
            for j in range(len(self.output_l_network.network)):
                weighted_sum += output_deltas[j] * self.output_l_network.network[j].weights[i]
            
            # Multiply the weighted sum by the error with respect to the input
            hidden_deltas.append(weighted_sum * self.hidden_l_network.network[i].error_wrt_input())

        # Update output weights and biases
        for i, neuron in enumerate(self.output_l_network.network):
            for j in range(len(neuron.weights)):
                neuron.weights[j] += learning_rate * output_deltas[i] * hidden_output[j]
            neuron.bias += learning_rate * output_deltas[i]
  

        # Update hidden weights and biases
        for i, neuron in enumerate(self.hidden_l_network.network):
            for j in range(len(neuron.weights)):
                neuron.weights[j] += learning_rate * hidden_deltas[i] * input_data[j]
            neuron.bias += learning_rate * hidden_deltas[i]



    def calculate_total_error(self, dataset):
        total_error = 0
        for input_data, target_op in dataset:
            outputs = self.forward_hidden_op(input_data)
            total_error += sum(0.5 * (target_op[i] - outputs[i]) ** 2 for i in range(len(target_op)))
        return total_error




            
        # // TODO: hidden layer delta calculations using chain rule
        # for i in range(len(self.hidden_l_network.network)):
        #     d=0
        #     for j in range(len(self.output_l_network.network)):
        #         # error_at_hidden_layer[i] = self.hidden_l_network[i].total_error(target_inp[i])
        #         d=error_at_op[i]*self.output_l_network.network[j].weights[i]
        #     error_at_hidden_layer[i] = d*self.hidden_l_network.network[i].error_wrt_input()

        
        # for i in range(len(self.output_l_network.network)):
        #     for j in range(len(self.hidden_l_network.network[i].weights)):
        #         error_wrt_weight=error_at_op * self.output_l_network.network[i].error_wrt_weight(j)
        #         self.hidden_l_network.network[i].weight[j]-= 0.5*error_wrt_weight



        # for i in range(len(self.hidden_l_network.network)):
        #     for j in range(len(self.hidden_l_network.network[i].weights)):
        #         error_wrt_weight=error_at_hidden_layer * self.hidden_l_network.network[i].error_wrt_weight(j)
        #         self.output_l_network.network[i].weight[j]-= 0.5*error_wrt_weight





In [45]:
nn = all_network(2, 2, 2, hidden_weights=[0.15, 0.2, 0.25, 0.3], hidden_bias=0.35, output_weights=[0.4, 0.45, 0.5, 0.55], output_bias=0.6)

# dataset = [([0.05, 0.1], [0.01, 0.99])]

# for epoch in range(100):
#     nn.backPropogation([0.05, 0.1], [0.01, 0.99])
#     if epoch % 10 == 0:
#         print(f"Epoch {epoch}: Error = {nn.calculate_total_error(dataset)}")

for i in range(100):
    nn.backPropogation([0.05, 0.1], [0.01, 0.99])
    print(i, round(nn.calculate_total_error([[[0.05, 0.1], [0.01, 0.99]]]), 9))

0 0.280471447
1 0.261907623
2 0.242935433
3 0.223873213
4 0.205075318
5 0.186895748
6 0.169649683
7 0.153582241
8 0.138851411
9 0.125527038
10 0.113602735
11 0.103014893
12 0.093663058
13 0.085427754
14 0.078183997
15 0.07181033
16 0.066194128
17 0.061234149
18 0.056841263
19 0.052938092
20 0.04945807
21 0.046344249
22 0.043548059
23 0.041028127
24 0.038749202
25 0.036681218
26 0.034798481
27 0.033078983
28 0.031503823
29 0.03005672
30 0.028723603
31 0.027492278
32 0.026352138
33 0.025293932
34 0.02430956
35 0.023391912
36 0.022534727
37 0.021732474
38 0.02098025
39 0.020273704
40 0.019608955
41 0.018982539
42 0.018391352
43 0.017832608
44 0.017303799
45 0.016802663
46 0.016327154
47 0.01587542
48 0.015445776
49 0.015036693
50 0.014646775
51 0.014274747
52 0.013919441
53 0.013579791
54 0.013254813
55 0.012943606
56 0.012645338
57 0.012359242
58 0.012084609
59 0.011820785
60 0.011567162
61 0.011323176
62 0.011088305
63 0.010862061
64 0.010643992
65 0.010433676
66 0.010230717
67 0.010034

In [35]:
n=all_network(2,2,2,[0.15, 0.2, 0.25, 0.3],[0.4, 0.45, 0.5, 0.55],0.35,0.6)
output=n.forward_hidden_op([0.05, 0.1])
output
n.backPropogation([0.05, 0.1], [0.01, 0.99])

Hidden layer outputs: [np.float64(0.5932699921071872), np.float64(0.596884378259767)]
Output layer outputs: [np.float64(0.7513650695523157), np.float64(0.7729284653214625)]


TypeError: 'list' object cannot be interpreted as an integer

In [50]:
a=[0]*3
a

[0, 0, 0]

In [38]:
n=all_network(2,5,2,[0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6],[0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85],0.35,0.6)


In [15]:
output=n.forward_hidden_op([0.05, 0.1])
output

Hidden layer outputs: [np.float64(0.5932699921071872), np.float64(0.596884378259767), np.float64(0.6004882745269664), np.float64(0.6040813228693781), np.float64(0.6076631698328917)]
Output layer outputs: [np.float64(0.8911937165747111), np.float64(0.945503933223316)]


[np.float64(0.8911937165747111), np.float64(0.945503933223316)]

In [16]:
n.backPropogation([0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6],[0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85])

5 2
